In [ ]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
!pip3 install sklearn
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
!pip3 install textdistance
import textdistance

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

  Using cached https://files.pythonhosted.org/packages/5c/a1/273def87037a7fb010512bbc5901c31cfddfca8080bc63b42b26e3cc55b3/scikit_learn-0.23.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/12/ec3f2e203afa394a149911729357aa48affc59c20e2c1c8297a60f33f133/threadpoolctl-2.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c8/89/63171228d5ced148f5ced50305c89e8576ffc695a90b58fe5bb602b910c2/scipy-1.5.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a6/fc/36e52d0ae2aa502b211f1bcd2fdeec72d343d58224eabcdddc1bcb052db1/numpy-1.19.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fc/c9/f58220ac44a1592f79a343caba12f6837f9e0c04c196176a3d66338e1ea8/joblib-0.17.0-py3-none-any.whl


In [ ]:
bad = open('Рабочий стол/sents_with_mistakes.txt', encoding='utf8').read().splitlines()[:100]
true = open('Рабочий стол/correct_sents.txt', encoding='utf8').read().splitlines()[:100]

In [ ]:
print(bad[9])
print(true[9])

In [ ]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [ ]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [ ]:
# Ошибки вычисляются также, по словарю. Но ещё мы собираем частотности, 
# чтобы потом выбирать самое вероятное исправление.
# реализуем алгоритм Symspell, который как Норвиг, но другой
# Нас интересует всего один вид ошибки - удаление

corpus = open('Рабочий стол/wiki_data.txt', encoding='utf8').read()[:1000]

In [ ]:
# создаем словарь из корпуса, в котором будем потом искать слова
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [ ]:
WORDS.most_common(10)

In [ ]:
# фунцкия расчета вероятности появления слова в корпусе
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

In [ ]:
P('апофеоз')

In [ ]:
# Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, 
# которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return deletes

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
word = 'сонце'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [ ]:
splits[:10]

In [ ]:
deletes = [L + R[1:] for L, R in splits if R]
deletes[:10]

In [ ]:
print(edits1('девочка'))

In [ ]:
print(list(edits2('девочка')))

In [ ]:
# Для оценки используем будем использовать три метрики:

# 1) процент правильных слов;
# 2) процент исправленных ошибок
# 3) процент ошибочно исправленных правильных слов


In [ ]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

In [ ]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)
# Это было у Норвига, у нас пока не сильно прикольнее
# 0.7287712287712288
# 0.5057559478127398
# 0.23785459974732973

In [ ]:
# Давайте делать триграммную модель, берем наш корпус
corpus_wiki = [['<start>', '<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]
# функция, которая делает n-граммы
def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [ ]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus_wiki:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence,2))
    trigrams.update(ngrammer(sentence,3))
    
# эта функция предсказатель ошибок    
def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1
    
# Напишем функцию, которая принимает слово
# и находит ближайшее к нему в словаре с помощью косисного расстояния
# и честной метрики редактирования.   
def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    # ваш код здесь
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    sims = Counter()
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup,topn, metric=metric)
    return closest

In [ ]:
# оцените качество также как и раньше
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0



for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    word_pairs = [('<start>', '<start>')] + word_pairs
    pred_sent = []
    for j in range(1, len(word_pairs)):
        
        pred = None
        
        # проверяем, что слова нет в словаре, чтобы не исправлять все слова
        if not predict_mistaken(word_pairs[j][1], WORDS):
            pred = word_pairs[j][1]
            
        
        else:
            # находим кандидатов для исправления
            predicted = get_closest_hybrid_match(word_pairs[j][1], X, vec)
        
            # берем предыдущее слово для контекста
            prev_word = word_pairs[j-1][1]
            # это предыдущее предыдущего для триграммы
            prev_prev_word = word_pairs[j-2][0]
        
            # если у нас нет в модели такого слова, то мы не сможем вероятность посчитать
            # остается только взять первое по близости
            if prev_word not in unigrams:
                pred = predicted[0][0]
            # ?
            if prev_prev_word not in unigrams:
                pred = predicted[0][0]
            
        
            else:
                lm_predicted = []
                for word, m in predicted:
                    bigram = ' '.join([prev_word, word])
                    trigram = ' '.join([prev_prev_word, prev_word, word])
                    lm_predicted.append((word, (m)*((trigrams[trigram]/bigrams[bigram]))))

                
                
                if lm_predicted:

                    pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]

        
        if pred is None:
            pred = word_pairs[j][1]
        

        
        if pred == word_pairs[j][0]:
            correct += 1
        else:
            mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
        total += 1
            
        if word_pairs[j][0] == word_pairs[j][1]:
            total_correct += 1
            if word_pairs[j][0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word_pairs[j][0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)